In [ ]:
# env setup
from tqdm import tqdm
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [26]:
# seed random

import numpy as np
import random

try:
    RND
except NameError:
    RND = 123

np.random.seed(RND)
random.seed(RND)

In [14]:
# create output dir
try:
    if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)
except NameError:
    pass

In [ ]:
# create tensorboard logs dir
try:
    if not os.path.isdir(TENSORBOARD_DIR): os.makedirs(TENSORBOARD_DIR)
except NameError:
    pass

In [1]:
# create models checkpoint dir
try:
    if not os.path.isdir(MODEL_CHECKPOINT_DIR): os.makedirs(MODEL_CHECKPOINT_DIR)
except NameError:
    pass

In [2]:
# reading geotiff

import rasterio, warnings, numpy as np

def read_geotiff(path):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        with rasterio.open(path) as src:
            b, g, r, nir = src.read()
            return np.dstack([r, g, b, nir])

In [ ]:
# list files in directory

import os

def list_files(src_dir):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(src_dir):
        f.extend(filenames)
    return f

In [ ]:
# display model as svg
def model_as_svg(model):
    from IPython.display import SVG
    from keras.utils.vis_utils import model_to_dot
    return SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [2]:
# data generation
def generate_batch(n_samples,
                   batch_index,
                   X_files,
                   Y_tags_misc,
                   Y_tags_weather,
                   images_dir,
                   image_shape):

    Xs = np.zeros((n_samples, *image_shape), dtype=np.float32)

    batches_in_X = int(len(X_files) / n_samples)
    batch_index %= batches_in_X

    X_files_batch = X_files[batch_index * n_samples:
                            batch_index * n_samples + n_samples]
    i = 0
    for f in X_files_batch:
        Xs[i] = np.fromfile(
            images_dir + '/' + f, dtype=np.float32).reshape(image_shape)
        i += 1
        
    if Y_tags_misc is None or Y_tags_weather is None:
        
        return Xs
        
    else:

        Y_tags_misc_batch = Y_tags_misc[batch_index * n_samples:
                                        batch_index * n_samples + n_samples]

        Y_tags_weather_batch = Y_tags_weather[batch_index * n_samples:
                                              batch_index * n_samples + n_samples]

        return (Xs, {
            'tags_misc': Y_tags_misc_batch,
            'tags_weather': Y_tags_weather_batch
        })